(raga-recognition)=
# Automatic raga recognition

Automatic raga recognition works intend to identify the raga that is performed in an Indian Art Music recording. Historically, raga recognition approaches have been based on melodic features from the singing voice, trying to identify relationships between the sung patterns and scales. More recently, this problem is being addressed by passing the said melodic features through DL models that learn, more efficiently the relationships within the data and learn to discriminate between the different ragas in a collection of data {cite}`ross_raga_2017, madhusudhan_raga_2019, john_raga_2020, clayton_raga_2022`.

This is a core task within the melodic analysis of Indian Art Music, and most of the research on this topic builds on top of the Dunya Carnatic and Hindustani corpora {cite}`iam_corpora`.

In [ ]:
## Importing compiam to the project
import compiam

# Import extras and supress warnings to keep the tutorial clean
import numpy as np
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

Let's first list the available tools for the task of raga recognition.

In [ ]:
compiam.melody.raga_recognition.list_tools()

We already see that the available tool appears with ``*`` at the end of the name, which indicates that it may be loaded using the ``compiam.load_models()`` wrapper. Observing [the entry of this tool in the `compiam` documentation](https://mtg.github.io/compIAM/source/melody.html#module-compiam.melody.raga_recognition.deepsrgm), we note that an optional dependency is needed to properly load this tool. Let's install it:

In [ ]:
%pip install torch==1.8.0

DEEPSRGM {cite}`madhusudhan_raga_2019` is a DL-based model that uses melodic features to automatically identify the raga from an Indian Art Music recording. In the original paper, the authors use the Indian Art Music Raga Dataset {cite}`raga_dataset`, and report accuracy higher than 95%.

In [ ]:
# Importing a tool for automatic recognition
from compiam.melody.raga_recognition import DEEPSRGM

# This tools has * at the end so we can load the pre-trained instance
deepsrgm = compiam.load_model("melody:deepsrgm")

In [ ]:
# Showing raga mapping
pprint(deepsrgm.mapping)

Therefore, the provided pre-trained model has been trained targetting to the ragas included in this mapping. Let's select some examples that are tagged with raga instances included in the default mapping to evaluate this tool and showcase how it works.

In [ ]:
# Initialize saraga Carnatic on our example audio folder
saraga_carnatic = compiam.load_dataset(
    "saraga_carnatic", data_home="./../audio/mir_datasets/"
)

```{note}
Please note that to save space, we are not loading the entire dataset but just a single recording. However, we can still use the dataset loader, which in this case will properly work as long as we only retrieve the data for this particular recording.
```

The recording we have included beforehand for tutoring purposes is *Sri Raghuvara Sugunaalaya*, performed by Sanjay Subrahmanyan, at Vani Mahal. We will load this track by first parsing its corresponding id in the `mirdata` dataloader (we do that beforehand). The said identifier is: ``109_Sri_Raghuvara_Sugunaalaya``.

In [ ]:
# We first load the {ids: tracks} dict for Saraga Carnatic
saraga_tracks = saraga_carnatic.load_tracks()

# Now we can get a specific track
track_data = saraga_tracks["109_Sri_Raghuvara_Sugunaalaya"]
print("This recording includes raaga:", track_data.metadata["raaga"][0]["name"])
print("This recordings has unique Dunya ID:", track_data.metadata["raaga"][0]["uuid"])

**Nice!** Raga Bhairavi is both in this recording and included in the mapping of DEEPSRGM, therefore considered during the training process. We first need to extract the features from an audio recording.

In [ ]:
# Computing features for the given recording
feat = deepsrgm.get_features(track_data.audio_path)

Finally, we can pass the computed features through the pre-trained model to perform inference and try to automatically recognise the raga performed in the recording.

```python
deepsrgm.predict(feat)
```

**Oops! Why is this line of code not running?** This ``.predict()`` method runs inference with the DEEPSRGM model using the passed features. As you may now, DL models tend to occupy an important amount of memory from your computer to run, especially in the training stage. Some light-weight models can be run from a conventional laptop, but in some other cases you might need a machine with enough power to run your models and perform inference. 

```{tip}
As mentioned in the [introduction](google-collab), Google Collab has GPU and TPU access which may allow you running certain models that are too big for your machine.
```

In [ ]:
## LOAD THIS NOTEBOOK INTO A GOOGLE COLLAB SESSION, UNCOMMENT AND RUN THIS LINE:
# deepsrgm.predict(feat)